In [ ]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", None, "display.max_columns", None)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Load the filtered admissions file which contains heart failure patients

df_admissions = pd.read_csv('ADMISSIONS_FILTERED.csv')
list_patients = df_admissions['SUBJECT_ID'].unique()
len(list_patients)

In [ ]:
#Read the CHARTEVENTS file for the required vital signs (please note that all the files have been created using the default 
#directory and the user needs to change accordingly).

list_vital = [224167, 227242, 227243, 227537, 227538, 224751, 
              224639, 224643, 220045, 220046, 220047, 220050, 
              220051, 220052, 220056, 220058, 220224, 220227, 
              226512, 226707, 220277, 223769, 223770, 442, 
              443, 8440, 51, 8368, 52, 8555, 6701 , 6702, 53, 
              490, 646, 733, 3580, 762, 763, 3693, 5820, 8554, 
              211, 3494, 1394]

columns = ['SUBJECT_ID', 'HADM_ID', 'ITEMID', 'CHARTTIME', 'VALUENUM', 'VALUEUOM']
df = pd.read_csv(r"CHARTEVENTS.csv", chunksize=10000000, usecols=columns,
                 low_memory=False)

i = 1
df_vitals=pd.DataFrame()

for data in df:
    print("Chunk no", i)
    data = data[data['SUBJECT_ID'].isin(list_patients) & data['ITEMID'].isin(list_vital)]
    data.drop_duplicates(keep=False, inplace=True)
    df_vitals=df_vitals.append(data,ignore_index=True)
    i=i+1

df2 = pd.read_csv('D_ITEMS.csv')
df_vitals = pd.merge(df_vitals,df2[['ITEMID','LABEL']],on='ITEMID', how='left')
df_vitals.to_csv('vitals_filtered_2.csv', index=False)    

In [ ]:
#The above mentioned dataframe has been saved and then read again to avoid going through the whole process again and again

import pandas as pd
df = pd.read_csv('vitals_filtered_2.csv')

In [ ]:
#Deleted column that is not required

del df['VALUEUOM']

In [ ]:
#Start creating the required dataframe

df_events_sub = df[df['CHARTTIME'] == df.groupby(['SUBJECT_ID', 'HADM_ID'])['CHARTTIME'].transform('min')] 
df_events_sub = df_events_sub.rename(columns= {'CHARTTIME':'CHARTTIME_START'})
df_events_sub = df_events_sub[['SUBJECT_ID','HADM_ID','CHARTTIME_START']]
df = df.merge(df_events_sub, how = 'left', right_on = ['SUBJECT_ID','HADM_ID'], left_on = ['SUBJECT_ID','HADM_ID']) 

In [ ]:
#Convert date_columns

df['CHARTTIME_START'] = pd.to_datetime(df['CHARTTIME_START']) 
df['CHARTTIME'] = pd.to_datetime(df['CHARTTIME'])
df['REL_DAY'] = (df['CHARTTIME']-df['CHARTTIME_START']).dt.days
del df['CHARTTIME']
del df['CHARTTIME_START']

In [ ]:
#Create a new dataframe for manipulation

df2 = df.groupby(['SUBJECT_ID', 'HADM_ID', 'REL_DAY','ITEMID', 'LABEL'], as_index=False).mean('VALUENUM')

In [ ]:
#Save this dataframe to avoid repeating the whole process

df2.to_csv('vitals_demo_2.csv', index=False)

In [ ]:
#Restructure the dataframe to start creating the feature matrix and save the final dataframe

df_restructured = df2.pivot_table(index=["SUBJECT_ID", "HADM_ID", "REL_DAY"], columns=["LABEL"],values="VALUENUM").reset_index()
df_restructured.to_csv('vitals_restructured_2.csv', index=False)

In [ ]:
#Start reading the filtered file again to create death time in ICU and save new dataframe with last chart time

df_restructured = pd.read_csv('vitals_restructured_2.csv')
df_max_day=(df_restructured.loc[df_restructured.groupby(['GROUP_ID'])['REL_DAY'].idxmax()])
df_max_day_charttime = pd.merge(df_max_day, df_chart_max[['GROUP_ID','CHARTTIME']],on='GROUP_ID', how='left')
df_max_day_charttime['CHARTTIME'] = pd.to_datetime(df_max_day_charttime['CHARTTIME']).dt.date
df_max_day_charttime['CHARTTIME'] = pd.to_datetime(df_max_day_charttime['CHARTTIME']).dt.date
df_max_day_charttime = df_max_day_charttime.rename(columns={'CHARTTIME': 'DEATHTIME'})
df_max_day_charttime.to_csv('chart_time_last.csv', index=False)

In [ ]:
#Read the filtered admissions file and check it with death time

df_admissions = pd.read_csv('ADMISSIONS_FILTERED.csv')

df_max_day_charttime = df_max_day_charttime[['SUBJECT_ID', 'HADM_ID', 'GROUP_ID', 'REL_DAY', 'DEATHTIME',
       'ART Blood Pressure Alarm - High', 'ART Blood Pressure Alarm - Low',
       'Admission Weight (Kg)', 'Admit Wt', 'Arterial BP #2 [Diastolic]',
       'Arterial BP #2 [Systolic]', 'Arterial BP Mean', 'Arterial BP Mean #2',
       'Arterial BP [Diastolic]', 'Arterial BP [Systolic]',
       'Arterial Blood Pressure Alarm - High',
       'Arterial Blood Pressure Alarm - Low',
       'Arterial Blood Pressure diastolic', 'Arterial Blood Pressure mean',
       'Arterial Blood Pressure systolic', 'Arterial O2 Saturation',
       'Arterial O2 pressure', 'Daily Weight', 'Heart Rate',
       'Heart Rate Alarm - Low', 'Heart rate Alarm - High', 'Height',
       'Manual BP Mean(calc)', 'Manual BP [Diastolic]', 'Manual BP [Systolic]',
       'Manual Blood Pressure Diastolic Left',
       'Manual Blood Pressure Diastolic Right',
       'Manual Blood Pressure Systolic Left',
       'Manual Blood Pressure Systolic Right',
       'O2 Saturation Pulseoxymetry Alarm - High',
       'O2 Saturation Pulseoxymetry Alarm - Low',
       'O2 saturation pulseoxymetry', 'PAO2', 'SpO2', 'SpO2 Alarm [High]',
       'SpO2 Alarm [Low]', 'Temporary Pacemaker Rate', 'Weight Change']]

final_df = pd.merge(df_max_day_charttime, df_admissions,  how='left', left_on=['GROUP_ID','DEATHTIME'], right_on = ['GROUP_ID','DEATHTIME'])

df_admissions.to_csv('ADMISSIONS_FILTERED.csv', index=False)

In [ ]:
df_vitals = pd.read_csv('vitals_restructured_2.csv')
df_max_day_charttime=pd.read_csv('chart_time_last.csv')
df_adm = pd.read_csv('ADMISSIONS_FILTERED.csv')

df_max_day_charttime.columns = df_max_day_charttime.columns.str.replace('DEATHTIME', 'LAST_CHART_TIME')
df_max_day_charttime = df_max_day_charttime[['SUBJECT_ID', 'HADM_ID', 'GROUP_ID', 'REL_DAY','LAST_CHART_TIME']]
res = df_max_day_charttime.merge(df_adm, how='inner', left_on=['GROUP_ID', 'LAST_CHART_TIME'], right_on=['GROUP_ID', 'DEATHTIME'])

#Remove duplicate columns after merge operation
res=res.drop(['SUBJECT_ID_y', 'HADM_ID_y', 'MARITAL_STATUS', 'GENDER', 'AGE'], axis=1)
res = res.rename(columns={'SUBJECT_ID_x': 'SUBJECT_ID', 'HADM_ID_x': 'HADM_ID'})

#Check if the last chart time matches with the death time recorded in the hospital to ensure only the patients who died 
#in the hopspital ICU have been included in this study

res['LAST_CHART_TIME'].equals(res['DEATHTIME']) 

In [ ]:
final_df = df_vitals.merge(res, how='inner', left_on=['GROUP_ID', 'REL_DAY'], right_on=['GROUP_ID', 'REL_DAY'])
df2 = pd.merge(df_vitals, res, on=['GROUP_ID','REL_DAY'])

#Create death chart csv file
res.to_csv('chart_death.csv', index=False)

In [ ]:
#Now start creating final dataframe that contains the actual death time flagged along with the vital signs

df_vitals = pd.read_csv('vitals_restructured_2.csv')
df_chart_death = pd.read_csv('chart_death.csv')
df = df_vitals.merge(df_chart_death, on=['GROUP_ID','REL_DAY'], how='outer')

#Rename the columns which changed due to duplication
df = df.rename(columns={'SUBJECT_ID_x': 'SUBJECT_ID', 'HADM_ID_x': 'HADM_ID'})

#Remove redundant columns created due to the merging process
df.drop(['SUBJECT_ID_y', 'HADM_ID_y'], axis=1, inplace=True)

#The following steps have taken to fill null values with 0
df['LAST_CHART_TIME'] = df['LAST_CHART_TIME'].fillna(0)
df['DEATHTIME'] = df['DEATHTIME'].fillna(0)
df['DEATH_IN_HOSPITAL'] = df['DEATH_IN_HOSPITAL'].fillna(0)

df.to_csv('chart_events_final.csv', index=False)